# Basics of Mobile Robotics, Project report 

The final project for EPFL's [**MICRO-452: Basics of Mobile Robotics**](https://edu.epfl.ch/coursebook/en/basics-of-mobile-robotics-MICRO-452) course. This project implements a complete navigation stack for a differential drive robot, featuring computer vision, graph-based path planning, sensor fusion (EKF), and reactive control.

**The Concept:** Inspired by the Disney movie WALL·E, the robot EVE must navigate through the interior of the spaceship Axiom - a complex and hazardous environment. Her mission is to reach, her love,  WALL·E as fast as possible, while avoiding a series of dangerous obstacles.
The map includes:

- Windows: Represented as static dark-blue polygonal regions that EVE must not cross. These areas act as boundaries or fatal zones, similar to voids in the spacecraft’s structure.

- Humans (“Passengers”): Large, drifting individuals who move unpredictably through the corridors. They function as dynamic obstacles that EVE must detect and avoid in real time.

![Example environment](images/warped.jpg)


## Group members & tasks repartition
- Tancrede Lamort de Gail: Vision, global navigation, motion control
- John Constantin: Vision, global navigation, motion control
- Marcus Edjolo: Local naviation, global navigation, motion control
- Yvan Barragan: Filtering, motion control


## 1. Physical Setup

The project tasks a Thymio II robot with navigating from a start pose to a goal pose while avoiding two distinct types of obstacles.

### 1.1 The Environment

The playground consist of a white rectangle of 130x92 cm delimitated by four Aruco Marker place on each of the 4 corners.

The environment presents a dual-layer challenge:

1.  **Static "Global" Obstacles (The Windows):**
    -   **Physicality:** Flat, dark blue polygonal cutouts.
    -   **Visibility:** _Invisible_ to the robot's onboard horizontal proximity sensors, but clearly visible to the overhead global camera.
    -   **Constraint:** The robot cannot drive over them. Avoiding them relies entirely on accurate global localization and path planning.
2.  **Ephemeral "Local" Obstacles (The Passengers):**
    -   **Physicality:** 3D physical objects (cylinders, blocks) roughly the size of the robot.
    -   **Visibility:** Visible to the robot's onboard sensors, but _ignored_ by the global camera mapping (or placed after mapping is complete).
    -   **Constraint:** These act as dynamic blockers. The robot must use local sensing to reactively avoid them without losing track of its global objective.

### 1.2 The Robot

The **Thymio II** is a differential-drive mobile robot.

-   **Sensors:** 5 front-facing and 2 rear-facing horizontal IR proximity sensors (for local avoidance).
-   **Comms:** Python API interfacing via USB/RF dongle.
-   **Markers:** An ArUco marker is mounted on top of the robot to facilitate high-precision global tracking (Pose: $x, y, \theta$) via the overhead camera.

### 1.3 The Camera

We're usging our smartphone mounted overhead to have a better quality and beging less sentistive to the environment. With the software "Camo studio" we are able the read the phone camera at $1920 \times 1080$, RGB, 30 FPS and transmit to the program.



## 2. Vision System
The vision systeme serves as the "GPS" of the system, responsible for:

1.  **Initial Mapping:** Detecting static obstacle polygons and the goal area.
2.  **Live Tracking:** Providing absolute position estimates to correct the robot's drifting odometry.


### 2.1 System Architecture

The vision pipeline operates in two distinct phases:

#### 2.1.1 Initialization Phase (Static Mapping)
During startup, the system captures a single reference frame and extracts the complete static environment:
- Calibration markers define the workspace boundaries
- Static obstacles (the "spacecraft windows") are segmented and converted to polygonal representations
- Goal location (WALL·E's position) is identified
- Visibility graph is pre-computed for path planning

#### 2.1.2 Runtime Phase (Real-Time Localization)
During navigation, the system continuously:
- Captures frames at approximately 10 Hz
- Detects (EVE's position) the robot's ArUco marker
- Computes the robot's pose $(x, y, \theta)$
- Provides measurements to the Extended Kalman Filter for sensor fusion

### 2.2 Coordinate Systems and Perspective Transformation

The vision system must bridge two distinct coordinate frames:

**Camera Frame:** The raw image captured by the overhead camera, measured in pixels with origin at top-left, where coordinates are $(u, v) \in [0, 1920] \times [0, 1080]$.

**World Frame:** The robot's navigation space, measured in centimeters with origin at bottom-left of the workspace, where coordinates are $(x, y) \in [0,132.5 ] \times [0, 92.5]$.


  <figure style="margin: 0; text-align: center;">
    <img src="images/warmup_frame.jpg" alt="Raw BGR image" width="500"/>
    <figcaption>Raw BGR image</figcaption>
  </figure>



#### 2.2.1 Perspective Correction

Due to the camera's mounting angle and lens distortion, the raw image exhibits perspective distortion. We use a **homography transformation** to warp the image into an orthographic (bird's-eye) view:

$$
\begin{bmatrix} u' \\ v' \\ w \end{bmatrix} = \mathbf{H} \begin{bmatrix} u \\ v \\ 1 \end{bmatrix}
$$

where $\mathbf{H} \in \mathbb{R}^{3 \times 3}$ is the perspective transformation matrix computed via OpenCV's `getPerspectiveTransform()` using the four corner calibration markers as control points.

The final pixel coordinates after perspective correction are:
$$
x_{\text{px}} = \frac{u'}{w}, \quad y_{\text{px}} = \frac{v'}{w}
$$

#### 2.2.2 Metric Conversion

After warping, we convert from the rectified pixel space to world coordinates (cm):

$$
x_{\text{cm}} = \frac{x_{\text{px}}}{\alpha_x}, \quad y_{\text{cm}} = H_{\text{map}} - \frac{y_{\text{px}}}{\alpha_y}
$$

where:
- $\alpha_x = \frac{W_{\text{px}}}{W_{\text{cm}}}$ and $\alpha_y = \frac{H_{\text{px}}}{H_{\text{cm}}}$ are pixel-to-centimeter scaling factors
- $H_{\text{map}}$ is the map height (flipping y-axis from image convention)


  <figure style="margin: 0; text-align: center;">
    <img src="images/warped.jpg" alt="Warped image" width="500"/>
    <figcaption>Warped image</figcaption>
  </figure>




### 2.3 Environment Segmentation

#### 2.3.1 Color-Based Segmentation
We segment obstacles and the goal using **HSV color space** thresholding (finding experimentally), which is more robust to lighting variations than RGB:

**Static Obstacles (Dark Blue):**
$$
\text{Mask}_{\text{obstacle}} = \{(h,s,v) \mid 90° \leq h \leq 165°, \, 40 \leq s \leq 255, \, 80 \leq v \leq 255\}
$$

**Goal Region (Red):**
Due to red's wraparound in HSV (0° and 360° are both red), we use a dual-range mask:
$$
\text{Mask}_{\text{goal}} = \text{Mask}_{\text{red1}} \cup \text{Mask}_{\text{red2}}
$$
where:
- $\text{Mask}_{\text{red1}}: 0° \leq h \leq 15°$
- $\text{Mask}_{\text{red2}}: 165° \leq h \leq 180°$


#### 2.3.2 Morphological Filtering
To eliminate noise and fill gaps in the segmented regions, we apply morphological operations with a $7 \times 7$ kernel:
1. **Opening** (erosion → dilation): Removes small noise blobs
2. **Closing** (dilation → erosion): Fills internal holes in obstacles

<div style="display: flex; gap: 20px; justify-content: center; align-items: flex-start;">

  <figure style="margin: 0; text-align: center;">
    <img src="images/obstacle_mask.jpg" alt="Obstacle mask" width="300"/>
    <figcaption>Obstacle mask</figcaption>
  </figure>

  <figure style="margin: 0; text-align: center;">
    <img src="images/target_mask.jpg" alt="Target mask" width="300"/>
    <figcaption>Target mask</figcaption>
  </figure>
</div>

### 2.5 Configuration Space Construction

#### 2.5.1 Polygon Approximation

Raw contours from segmentation are noisy and contain hundreds of vertices. We simplify them into approximated polygons via cv2's `approxPolyDP` function, which uses the **Douglas-Peucker algorithm** with an error proportional to contour length:

$$
\epsilon = 0.015 \cdot \text{arcLength}(\text{contour}) \\
P = \text{cv2.approxPolyDP}(C, \epsilon)
$$

This produces polygons with 4-12 vertices while preserving essential shape features.

#### 2.5.2 C-Space Inflation
To account for the robot's physical dimensions, we **buffer** each obstacle polygon by the robot's radius plus a safety margin:

$$
\mathcal{O}_{\text{buffered}} = \mathcal{O} \oplus B_r
$$

where:
- $\mathcal{O}$ is the original obstacle polygon
- $B_r$ is a disk of radius $r = r_{\text{robot}} + \text{padding}$ (≈ 5.5 cm)
- $\oplus$ denotes Minkowski sum (implemented via Shapely's `buffer()` with mitre join style, and mitre chopping to avoid excessively pointy edges)

This operation effectively reduces the robot to a point in configuration space, simplifying collision checking.

#### 2.5.3 Vertex Merging
The buffering operation can introduce redundant vertices on long straight edges. To reduce graph complexity, we merge vertices closer than a threshold $d_{\text{min}} = 6.0$ cm:

$$
\text{if } \|v_i - v_{i+1}\| < d_{\text{min}}, \quad v_{\text{merged}} = \frac{v_i + v_{i+1}}{2}
$$

### 2.6 Visibility Graph Construction

The visibility graph $\mathcal{G} = (\mathcal{V}, \mathcal{E})$ encodes all collision-free straight-line paths through the environment. We leverage two Python libraries for this task: **Shapely** for geometric operations (polygon intersection, line-polygon collision detection) and **NetworkX** for graph representation and manipulation.

#### 2.6.1 Node Addition
We begin by constructing the node set $\mathcal{V}$, which includes all vertices from the buffered obstacle polygons plus the goal location:

$$
\mathcal{V} = \mathcal{V}_{\text{obstacles}} \cup \{v_{\text{goal}}\}
$$

Each node $v_i$ is added to a NetworkX graph with an attribute `pos` storing its $(x, y)$ coordinates in centimeters.

#### 2.6.2 Obstacle Edge Insertion
Before computing visibility between arbitrary node pairs, we first add all **obstacle boundary edges** to the graph. For each obstacle polygon with vertices $\{v_0, v_1, \ldots, v_{n-1}\}$, we connect consecutive vertices:

$$
\mathcal{E}_{\text{obstacle}} = \{(v_i, v_{i+1 \mod n}) \mid i = 0, \ldots, n-1\}
$$

Each edge is weighted by Euclidean distance:
$$
w(v_i, v_j) = \|v_i - v_j\|_2 = \sqrt{(x_j - x_i)^2 + (y_j - y_i)^2}
$$

This ensures that paths can traverse obstacle perimeters when necessary.

#### 2.6.3 Visibility Edge Computation
Next, we test visibility between all pairs of nodes. Two nodes $v_i, v_j$ are connected by an edge if:
1. They are not already connected (to avoid duplicate obstacle edges)
2. The line segment $\overline{v_i v_j}$ does not intersect any obstacle interior

We implement visibility checking using Shapely's geometric predicates. The test verifies that the line segment neither crosses through nor lies entirely within any obstacle:

$$
\text{visible}(v_i, v_j) = \begin{cases}
\text{True} & \text{if } \overline{v_i v_j} \cap \mathcal{O}_k = \emptyset \, \forall k \text{ and } \overline{v_i v_j} \not\subset \mathcal{O}_k \, \forall k \\
\text{False} & \text{otherwise}
\end{cases}
$$

The second condition is critical for concave obstacles - a line segment could lie entirely within a U-shaped obstacle without technically "intersecting" its boundary.

**Shrinking Heuristic:** To avoid false positives from floating-point precision issues at polygon vertices, we test a **slightly shortened** version of each line segment:

$$
\overline{v_i v_j}_{\text{test}} = \{v_i + 0.001 \cdot (v_j - v_i) \to v_i + 0.999 \cdot (v_j - v_i)\}
$$

This prevents spurious intersections when paths graze obstacle corners.

#### 2.6.4 Boundary Cropping
After constructing the full graph, we perform a final validation step: **remove any nodes that lie outside the map boundaries**. Due to the buffering operation, some obstacle vertices may extend beyond the physical workspace limits $[0, W_{\text{map}}] \times [0, H_{\text{map}}]$.

We identify and remove all such nodes:
$$
\mathcal{V}_{\text{invalid}} = \{v_i \in \mathcal{V} \mid x_i < 0 \text{ or } x_i > W_{\text{map}} \text{ or } y_i < 0 \text{ or } y_i > H_{\text{map}}\}
$$

$$
\mathcal{V}_{\text{final}} = \mathcal{V} \setminus \mathcal{V}_{\text{invalid}}
$$

NetworkX automatically removes all edges connected to deleted nodes, ensuring graph integrity.

#### 2.6.5 Base Graph Storage
The result of this process is the **base visibility graph**, which contains only obstacle vertices and the goal node. This graph is computed once during initialization and stored as `self.base_graph`. The robot's start position is intentionally excluded at this stage to enable dynamic replanning after kidnapping events.




### 2.7 Dynamic Graph Augmentation

Since the robot's start position changes after each kidnapping event, the visibility graph must be **dynamically updated**:

1. **Base Graph:** Contains only obstacle vertices and the goal (precomputed once)
2. **Runtime Augmentation:** When the robot is relocated:
   - Add a new start node $v_{\text{start}}$ at the robot's current pose
   - Test visibility from $v_{\text{start}}$ to all existing nodes
   - Connect $v_{\text{start}}$ to all visible nodes with weighted edges
   - Run A* search on the augmented graph

This strategy avoids recomputing the entire $O(n^3)$ visibility graph (where $n \approx 40$ nodes) after every kidnapping - we only perform $O(n^2)$ visibility tests from the new start node.

 

### 2.8 ArUco Marker Detection and Pose Estimation

#### 2.8.1 Pose Computation
Given the four corners of the detected marker $\{C_0, C_1, C_2, C_3\}$ (ordered as Top-Left, Top-Right, Bottom-Right, Bottom-Left), we compute:

**Position** (centroid):
$$
x = \frac{1}{4}\sum_{i=0}^{3} C_{i,x}, \quad y = \frac{1}{4}\sum_{i=0}^{3} C_{i,y}
$$

**Orientation** (from marker's left edge):
$$
\theta = \text{atan2}(C_{0,y} - C_{3,y}, \, C_{0,x} - C_{3,x})
$$

This orientation vector points along the robot's forward direction, as the marker is mounted with its left edge aligned with the robot's heading.

### 2.8 Measurement Uncertainty

The camera provides pose estimates $(x, y, \theta)$ with empirically measured covariance:

$$
\mathbf{R}_{\text{camera}} = \begin{bmatrix}
0.00119 & 0 & 0 \\
0 & 0.00179 & 0 \\
0 & 0 & 0.000061
\end{bmatrix}
$$

This translates to standard deviations of approximately:
- Position: $\sigma_x \approx 3.4$ mm, $\sigma_y \approx 4.2$ mm
- Orientation: $\sigma_\theta \approx 0.45°$

These values were obtained through repeated measurements of a stationary robot and used to tune the Extended Kalman Filter.

## 3. Path Planning

Once the vision system has constructed a visibility graph of the environment, the robot needs to compute the shortest collision-free path from its current position to the goal. We implement this using the **A\* (A-star) search algorithm**.


### 3.1 Implementation Details

The implementation uses a **priority queue** (min-heap) to efficiently retrieve the node with the lowest $f$-cost at each iteration:

**NetworkX Integration:** Our visibility graph is stored as a `networkx.Graph` object where:
- Nodes have attribute `pos: Point` (world coordinates)
- Edges have attribute `weight: float` (Euclidean distance)

This abstraction allows A\* to operate independently of the graph construction method.

### 3.2 Waypoint Generation

The output of A\* is a sequence of node indices:
$$
\text{path} = [n_{\text{start}}, n_1, n_2, \ldots, n_{k-1}, n_{\text{goal}}]
$$

We convert these to world-space waypoints by extracting the `pos` attribute:
$$
\text{waypoints} = [(x_0, y_0), (x_1, y_1), \ldots, (x_k, y_k)]
$$

The controller then navigates sequentially through these waypoints (nodes on green line in the figure below), moving to the next one once within a threshold distance of the current target.

  <figure style="margin: 0; text-align: center;">
    <img src="images\visu.jpg" alt="Optimal path" width="500"/>
    <figcaption>Optimal path</figcaption>
  </figure>



# 4. Local Navigation and Control

While the vision system and path planner provide a global roadmap (Section 1), the robot must execute this plan in the real world using onboard sensors and actuators. This layer handles two critical tasks: (1) **waypoint tracking**—following the planned path, and (2) **reactive obstacle avoidance**—responding to dynamic obstacles not captured during mapping.

---

## 4.1 Control Architecture (Finite State Machine)

The control system operates as a predictable **hierarchical state machine** with two primary operational states. The transition logic manages the stability requirements of both global path-following and local evasion.

### 4.1.1 State: NAVIGATING

This state encompasses both goal-seeking behavior (tracking waypoints) and the smooth return phase after avoidance.

**Objective:** Drive toward the current waypoint or smoothly blend control to return to waypoint tracking.

#### A. Waypoint Tracking (Pure Navigation)

This control law is used when no avoidance maneuver is in progress ($\alpha=0$).

**Control Law:** Proportional control on heading error with speed regulation based on alignment:

$$
\begin{align}
\theta_{\text{error}} &= \text{normalize}\left(\text{atan2}(y_{\text{target}} - y, x_{\text{target}} - x) - \theta\right) \\
\omega &= K_p \cdot \theta_{\text{error}} \\
v &= v_{\max} \cdot \max(0, \cos(\theta_{\text{error}}))
\end{align}
$$

where:
* $K_p = 120$ is the proportional gain for angular velocity
* $v_{\max} = 125$ Thymio units
* $\text{normalize}(\cdot)$ wraps angles to $[-\pi, \pi]$

The cosine term elegantly handles the speed-accuracy tradeoff: when the robot is well-aligned with the target ($\theta_{\text{error}} \approx 0$), it drives at full speed. When misaligned ($|\theta_{\text{error}}| \approx \pi/2$), it stops forward motion and rotates in place.

**Differential Drive Kinematics:**
$$
\begin{align}
v_L &= v - \omega \\
v_R &= v + \omega
\end{align}
$$

#### B. Decay Phase (Smooth Return)

This control law is used for duration $T = 4.0$ seconds after obstacle clearance ($\alpha > 0$). It achieves a smooth transition back to pure navigation control.

**Control Law (Linear Blending):**

$$
\begin{align}
\alpha(t) &= \max\left(0, 1 - \frac{t}{T}\right) \quad \text{(decay factor)} \\
v_L &= \alpha \cdot v_{\max} + (1 - \alpha) \cdot v_{L,\text{nav}} \\
v_R &= \alpha \cdot v_{\max} + (1 - \alpha) \cdot v_{R,\text{nav}}
\end{align}
$$

where $v_{L/R,\text{nav}}$ are the wheel speeds output by the Waypoint Tracking controller. This linear interpolation prevents oscillations and ensures the robot smoothly reorients toward the path.

### 4.1.2 State: AVOIDING

**Trigger:** Any front proximity sensor exceeds threshold ($\mathbf{400}$ units).

**Objective:** Steer away from obstacle using a computationally trivial reactive controller.

**Control Law (Differential Steering):**

$$
\tau = w_{\text{outer}}(s_0 - s_4) + w_{\text{inner}}(s_1 - s_3)
$$

where:
* $s_i$ are the five front proximity sensor readings (indexed left to right).
- $w_{\text{outer}} = 1.0$, $w_{\text{inner}} = 1.7$ are tuned weights
* $\tau$ is the turn bias (positive = turn right, negative = turn left).

The motor commands are:
$$
\begin{align}
v_L &= v_{\text{avoid}} + k \cdot \tau \\
v_R &= v_{\text{avoid}} - k \cdot \tau
\end{align}
$$

with gain $k = 0.025$.

**Adaptive Weighting:** If the left-right sensor difference is too small ($|s_1 - s_3| < 30$), indicating a symmetric frontal obstacle, the system adapts by setting $w_{\text{inner}} = 0$ and increasing $w_{\text{outer}} = 2.3$. This forces the robot to commit to a sharp turn, preventing it from getting stuck on a head-on collision.

---

## 4.2 Waypoint Advancement and Design Rationale

### Waypoint Advancement

The robot monitors its Euclidean distance to the current waypoint:
$$
d = \sqrt{(x_{\text{waypoint}} - x)^2 + (y_{\text{waypoint}} - y)^2}
$$

The waypoint is considered reached, and the controller advances to the next one in the sequence, when $d < 1.5$ cm.

### Design Rationale

* **Why FSM?** Finite state machines provide predictable, debuggable behavior. By fusing the decay phase into the NAVIGATING state, we emphasize that the ultimate objective remains goal-oriented navigation, regardless of the intermediate control law.
* **Why Decay Phase?** Instantaneous switching from obstacle avoidance back to navigation causes "chattering" (the robot oscillates between states when near an obstacle). The decay phase acts as a **low-pass filter** on the control commands, smoothly restoring the proportional navigation signal.
* **Why Braitenberg-inspired Differential Steering?** While not globally optimal, this control law is computationally trivial (required for quick reflexes on the Thymio) and provides robust local evasion, effectively complementing the high-level global path planning.


## 5. Extended Kalman Filter (EKF) formulation for pose estimation

### 5.1 Principle of the Extended Kalman Filter

The Extended Kalman Filter (EKF) is a state estimator for **non-linear** dynamical systems. It assumes the system can be written as:

* **Nonlinear state (process) model:**
    $$\mathbf{x}_{k+1} = f(\mathbf{x}_k, \mathbf{u}_k) + \mathbf{w}_k$$

* **Nonlinear measurement model:**
    $$\mathbf{z}_k = h(\mathbf{x}_k) + \mathbf{v}_k$$

with:
* $\mathbf{x}_k$: state vector at time step $k$,
* $\mathbf{u}_k$: known control input (wheel speeds in our case),
* $\mathbf{z}_k$: measurement vector (camera-based pose),
* $\mathbf{w}_k \sim \mathcal{N}(\mathbf{0}, \mathbf{Q}_k)$: process noise,
* $\mathbf{v}_k \sim \mathcal{N}(\mathbf{0}, \mathbf{R}_k)$: measurement noise.

Following Becker’s notation in *Kalman Filter from the Ground Up*, the EKF linearizes the non-linear functions $f(\cdot)$ and $h(\cdot)$ around the current estimate using their Jacobians, and then applies the standard Kalman prediction–update loop to fuse odometry and camera measurements.



---

### 5.2 Nonlinear state-space model of the Thymio robot

We use a **unicycle-like kinematic model** with state:

$$
\mathbf{x}_k =
\begin{bmatrix}
x_k \\
y_k \\
\theta_k
\end{bmatrix}
$$

where:
* $x_k, y_k$ are the robot position in the global (camera) frame [cm],
* $\theta_k$ is the robot heading [rad].

The input is the pair of wheel linear velocities derived from Thymio’s internal velocity units:

$$
\mathbf{u}_k =
\begin{bmatrix}
v_{L,k} \\
v_{R,k}
\end{bmatrix}
$$



Given the wheelbase $L$, the linear and angular velocities are:

$$
V_k = \frac{v_{L,k} + v_{R,k}}{2}, \qquad
\omega_k = \frac{v_{R,k} - v_{L,k}}{L}
$$

With sampling period $\Delta t$, the discrete-time kinematics are:

$$
\begin{aligned}
x_{k+1} &= x_k + V_k \cos(\theta_k)\,\Delta t, \\
y_{k+1} &= y_k + V_k \sin(\theta_k)\,\Delta t, \\
\theta_{k+1} &= \theta_k + \omega_k\,\Delta t.
\end{aligned}
$$

This defines the nonlinear state transition function:

$$
\mathbf{x}_{k+1} = f(\mathbf{x}_k, \mathbf{u}_k) =
\begin{bmatrix}
x_k + V_k \cos(\theta_k)\,\Delta t \\
y_k + V_k \sin(\theta_k)\,\Delta t \\
\theta_k + \omega_k\,\Delta t
\end{bmatrix}
$$

This is implemented as the `state_extrapolation_f` model of the Thymio robot.

---

### 5.3 EKF prediction step

#### 5.3.1 Linearization of the process model

To propagate the state uncertainty, the EKF uses the Jacobian of $f$ with respect to the state:

$$
\mathbf{F}_k = \left.\frac{\partial f}{\partial \mathbf{x}}\right|_{\mathbf{x}=\hat{\mathbf{x}}_{k|k},\, \mathbf{u}=\mathbf{u}_k}
$$

From the kinematic model, using $V_k$ as defined above, we obtain:

$$
\mathbf{F}_k =
\begin{bmatrix}
1 & 0 & -V_k \sin(\hat{\theta}_{k|k})\,\Delta t \\
0 & 1 & \;\;\;V_k \cos(\hat{\theta}_{k|k})\,\Delta t \\
0 & 0 & 1
\end{bmatrix}
$$

which matches the `jacobian_dF` function in the implementation.

To inject **input noise** into the state covariance, we also use the Jacobian of $f$ with respect to the input:

$$
\mathbf{G}_k = \left.\frac{\partial f}{\partial \mathbf{u}}\right|_{\mathbf{x}=\hat{\mathbf{x}}_{k|k},\, \mathbf{u}=\mathbf{u}_k}
$$

The chosen input mapping leads to:

$$
\mathbf{G}_k =
\begin{bmatrix}
\frac{1}{2}\Delta t \cos(\hat{\theta}_{k|k}) & \frac{1}{2}\Delta t \cos(\hat{\theta}_{k|k}) \\
\frac{1}{2}\Delta t \sin(\hat{\theta}_{k|k}) & \frac{1}{2}\Delta t \sin(\hat{\theta}_{k|k}) \\
-\frac{\Delta t}{L} & \frac{\Delta t}{L}
\end{bmatrix}
$$

which corresponds to the `jacobian_G` function.

#### 5.3.2 State and covariance prediction

The prediction step is defined as:

* **State extrapolation:**
    $$\hat{\mathbf{x}}_{k+1|k} = f(\hat{\mathbf{x}}_{k|k}, \mathbf{u}_k)$$

* **Covariance extrapolation:**
    $$\mathbf{P}_{k+1|k} = \mathbf{F}_k\,\mathbf{P}_{k|k}\,\mathbf{F}_k^\top + \mathbf{Q}_k$$

where $\mathbf{Q}_k$ is the process noise covariance. In our design, $\mathbf{Q}_k$ is dominated by the input noise (wheel speed uncertainty), with a smaller additional term capturing modeling errors (see Section 5.5).

---

### 5.4 EKF update step — full pose measurement

The overhead vision system provides a direct measurement of the robot pose:

$$
\mathbf{z}_k =
\begin{bmatrix}
x^{\text{cam}}_k \\
y^{\text{cam}}_k \\
\theta^{\text{cam}}_k
\end{bmatrix}
=
\mathbf{x}_k + \mathbf{v}_k
$$

with measurement noise $\mathbf{v}_k$.

This corresponds to a **linear measurement model**:

$$
h(\mathbf{x}_k) = \mathbf{x}_k, \qquad
\mathbf{z}_k = h(\mathbf{x}_k) + \mathbf{v}_k
$$

so the observation matrix is simply:

$$
\mathbf{H}_k = \frac{\partial h}{\partial \mathbf{x}} = \mathbf{I}_3
$$

The update step is then:

* **Innovation:**
    $$\mathbf{y}_k = \mathbf{z}_k - h(\hat{\mathbf{x}}_{k|k-1}) = \mathbf{z}_k - \hat{\mathbf{x}}_{k|k-1}$$

* **Innovation covariance:**
    $$\mathbf{S}_k = \mathbf{H}_k \mathbf{P}_{k|k-1} \mathbf{H}_k^\top + \mathbf{R}_k$$

* **Kalman gain:**
    $$\mathbf{K}_k = \mathbf{P}_{k|k-1} \mathbf{H}_k^\top \mathbf{S}_k^{-1}$$

* **State update:**
    $$\hat{\mathbf{x}}_{k|k} = \hat{\mathbf{x}}_{k|k-1} + \mathbf{K}_k \mathbf{y}_k$$

* **Covariance update (Joseph form):**
    $$\mathbf{P}_{k|k} = \left(\mathbf{I} - \mathbf{K}_k \mathbf{H}_k\right) \mathbf{P}_{k|k-1} \left(\mathbf{I} - \mathbf{K}_k \mathbf{H}_k\right)^\top + \mathbf{K}_k \mathbf{R}_k \mathbf{K}_k^\top$$

This form is numerically stable and keeps $\mathbf{P}_{k|k}$ symmetric and positive semi-definite.

---

### 5.5 Uncertainty modelling and calibration

All noise terms are modeled as zero-mean Gaussian random variables:

* **Process noise:** $\mathbf{w}_k \sim \mathcal{N}(\mathbf{0}, \mathbf{Q}_k)$
* **Measurement noise:** $\mathbf{v}_k \sim \mathcal{N}(\mathbf{0}, \mathbf{R}_k)$

The key design choice in our EKF is that both $\mathbf{Q}_k$ and $\mathbf{R}$ are **data-driven**:

* $\mathbf{Q}_k$ is mostly built from the experimentally measured variability of Thymio’s wheel speeds.
* $\mathbf{R}$ is obtained from the variability of the camera’s pose estimates.

#### 5.5.1 Input noise and process noise $\mathbf{Q}_k$

We consider two contributions:
1.  **Input-dependent noise** due to uncertain wheel speeds.
2.  **Additional model noise** to capture unmodeled effects.

##### (a) Experimental acquisition of wheel speed variances (Results in 5.8)

To characterize the wheel speed noise, the Thymio was placed **in the air** so that the wheels could spin freely without interacting with the ground. For a series of target speeds (in Thymio units), we:

1.  Commanded a constant target speed to the robot.
2.  Recorded a set of speed samples from each motor.
3.  Computed the sample variance of the measured speed for that target.

This procedure was repeated for several target speeds. During analysis we observed:

* A clear **positive correlation** between the magnitude of the command and the variance: higher speed commands produced larger variance in the measured speed.
* Above approximately **200 Thymio units**, the variance increased very sharply and became dominated by outliers and nonlinearities, making it unsuitable for building a simple model.
* We therefore **restricted the data used for modelling** to commands up to **120 Thymio units**, which we defined as the maximum operational speed for our application, and where the variance behaviour is still well-behaved and approximately linear.

Using these filtered data (up to 120 units), we fitted a **linear regression model** for each wheel:

$$
\operatorname{var}(u_L) \approx a_L\,|u_L| + b_L,
\qquad
\operatorname{var}(u_R) \approx a_R\,|u_R| + b_R
$$

where $u_L, u_R$ are the raw Thymio commands, and the parameters $a_L, a_R, b_L, b_R$ are obtained from the regression.

In the implementation these relationships are encoded as:

$$
\begin{aligned}
\operatorname{var}(u_{L,k}) &= \max\big(k_{vL}\,|u_{L,k}| + \text{var}_{vL,\text{base}},\; \text{var}_{vL,\text{base}}\big), \\
\operatorname{var}(u_{R,k}) &= \max\big(k_{vR}\,|u_{R,k}| + \text{var}_{vR,\text{base}},\; \text{var}_{vR,\text{base}}\big)
\end{aligned}
$$

where $k_{vL}, k_{vR}$ are slopes from the regression and $\text{var}_{(\cdot),\text{base}}$ enforce a minimum variance level.

These variances are then converted from Thymio units to physical units (cm/s) using the speed calibration factor $c$:

$$
\operatorname{var}(v_{L,k}) = c^2\,\operatorname{var}(u_{L,k}), \qquad
\operatorname{var}(v_{R,k}) = c^2\,\operatorname{var}(u_{R,k})
$$

which yields the **input noise covariance**:

$$
\boldsymbol{\Sigma}_{u,k} =
\begin{bmatrix}
\operatorname{var}(v_{L,k}) & 0 \\
0 & \operatorname{var}(v_{R,k})
\end{bmatrix}
$$

To propagate this noise into the state, we use the previously defined Jacobian $\mathbf{G}_k$:

$$
\mathbf{Q}_{\text{input},k} = \mathbf{G}_k \, \boldsymbol{\Sigma}_{u,k} \, \mathbf{G}_k^\top
$$

This construction explicitly encodes the observed behaviour: as the wheel speeds increase (within the modeled range up to 120 units), their variance increases, and this higher input variance is correctly projected into larger uncertainty in $(x,y,\theta)$.

##### (b) Additional process noise

On top of the input noise, we add a small **model noise** term to account for unmodelled dynamics (e.g., small slips, discretization errors):

$$
\mathbf{Q}_{\text{model}} =
\operatorname{diag}
\big(
\sigma_{x,\text{model}}^2,\;
\sigma_{y,\text{model}}^2,\;
\sigma_{\theta,\text{model}}^2
\big)
$$

The values $\sigma_{x,\text{model}}, \sigma_{y,\text{model}}, \sigma_{\theta,\text{model}}$ were selected through a series of test runs, but **kept relatively small** on purpose. Since most of the motion uncertainty is already captured by the input noise model $\mathbf{Q}_{\text{input},k}$, this additional term only provides a modest baseline of process noise and avoids over-inflating the covariance.

##### (c) Final process covariance

The final process noise used in the EKF is:

$$
\boxed{
\mathbf{Q}_k
=
\mathbf{Q}_{\text{input},k} + \mathbf{Q}_{\text{model}}
}
$$

This design keeps $\mathbf{Q}_k$ physically meaningful (driven by the measured Thymio behaviour) while still allowing some flexibility through the small model noise component.

---

#### 5.5.2 Measurement noise covariance $\mathbf{R}$

The measurement noise covariance $\mathbf{R}$ was calibrated using the **camera pose detection system** itself.

To do this, the robot was placed **completely still** at a fixed pose in the arena, and the overhead camera was used to record a set of pose measurements $\{\mathbf{z}_i\}_{i=1}^N$, each:

$$
\mathbf{z}_i =
\begin{bmatrix}
x_{\text{cam},i} \\
y_{\text{cam},i} \\
\theta_{\text{cam},i}
\end{bmatrix}
$$

From these samples, we computed the sample mean:

$$\bar{\mathbf{z}} = \frac{1}{N}\sum_{i=1}^N \mathbf{z}_i$$

and the **sample covariance**:

$$
\hat{\mathbf{R}} =
\frac{1}{N-1}
\sum_{i=1}^N
(\mathbf{z}_i - \bar{\mathbf{z}})
(\mathbf{z}_i - \bar{\mathbf{z}})^\top
$$

In the EKF we then use a diagonal approximation of this covariance,

$$
\mathbf{R} =
\begin{bmatrix}
\sigma_{x,\text{cam}}^2 & 0 & 0 \\
0 & \sigma_{y,\text{cam}}^2 & 0 \\
0 & 0 & \sigma_{\theta,\text{cam}}^2
\end{bmatrix}
$$


$$
\mathbf{R} =
\begin{bmatrix}
1.186 \times 10^{-3} & 0 & 0 \\
0 & 1.787 \times 10^{-3} & 0 \\
0 & 0 & 6.101 \times 10^{-5}
\end{bmatrix}
$$

where the variances are taken from the diagonal (assuming no correlation between variables for simplicity of the scope of this project) of $\hat{\mathbf{R}}$ (after converting angles to radians). This way, the filter’s trust in the camera directly reflects the actual variability of the vision system.



#### 5.5.3 Initial state and covariance

Finally, the EKF must be initialized with:
* $\hat{\mathbf{x}}_{0|0}$: an initial guess of the robot pose.
* $\mathbf{P}_{0|0}$: the initial estimation uncertainty.

Consistent with standard practice, we use:
* A reasonable initial pose estimate (e.g., from the first valid camera detection or a known start pose).
* A **relatively large diagonal covariance** $\mathbf{P}_{0|0}$ to express our initial lack of certainty and to allow the filter to adapt quickly to the first few measurements.

---

### 5.6 Summary

* The EKF is built on a **nonlinear Thymio kinematic model** for pose propagation and a **linear measurement model** (Option A: direct pose measurement from vision).
* The EKF design focuses on **data-driven uncertainty modelling**:
    * **Process Noise ($\mathbf{Q}_k$):** Dominated by input noise, where wheel speed variances are measured experimentally (Thymio in the air) and propagated to the state via the Jacobian $\mathbf{G}_k$.
    * **Measurement Noise ($\mathbf{R}$):** Obtained directly from the sample covariance of camera pose measurements while the robot is static.
* This results in a filter whose parameters are grounded in the actual behaviour of the robot and the vision system, ensuring that the fusion weights (Kalman Gain) optimally balance the trust between odometry and vision.

### 5.7 Implementation Overview (Pseudocode)

To complement the EKF design and its mathematical formulation, this section provides a high-level pseudocode view of how the filter and the Thymio motion model are implemented in the project. The goal is not to present executable code, but to illustrate how the software structure corresponds directly to the mathematical equations in Section 5.

---

#### 5.7.1 Thymio Model: Motion and Process Noise

The `ThymioModel` module encapsulates the nonlinear motion model, the calculation of Jacobians, and the generation of the data-driven process noise covariance matrix ($Q_k$) used by the EKF.

```text
CLASS ThymioModel

    FUNCTION StateExtrapolation(x_state, u_input, dt)
        // Implements the nonlinear kinematic model: x_{k+1} = f(x_k, u_k)
        // INPUT:  x_state (3x1), u_input (wheel speeds), dt (timestep)
        // OUTPUT: x_next (3x1)
        BEGIN
            V     ← (u_input.v_r + u_input.v_l) / 2
            omega ← (u_input.v_r - u_input.v_l) / L
            
            x_next[0] ← x_state[0] + V * COS(x_state[2]) * dt
            x_next[1] ← x_state[1] + V * SIN(x_state[2]) * dt
            x_next[2] ← x_state[2] + omega * dt

            RETURN x_next
        END FUNCTION

    FUNCTION JacobianDF(x_state, u_input, dt)
        // Linearization w.r.t. the state: F_k = ∂f/∂x
        BEGIN
            V     ← (u_input.v_r + u_input.v_l) / 2
            theta ← x_state[2]

            F ← IdentityMatrix(3)
            F[0, 2] ← -V * dt * SIN(theta)
            F[1, 2] ←  V * dt * COS(theta)

            RETURN F
        END FUNCTION

    FUNCTION JacobianG(x_state, dt)
        // Linearization w.r.t. the inputs: G_k = ∂f/∂u
        BEGIN
            theta ← x_state[2]
            
            // Map wheel speed noise into state space (x, y, theta)
            G ← Matrix(3, 2)
            G[0, 0] ← 0.5 * dt * COS(theta);  G[0, 1] ← 0.5 * dt * COS(theta)
            G[1, 0] ← 0.5 * dt * SIN(theta);  G[1, 1] ← 0.5 * dt * SIN(theta)
            G[2, 0] ← -dt / L;                G[2, 1] ← dt / L

            RETURN G
        END FUNCTION

    FUNCTION ComputeQ(x_state, u_input)
        // Data-driven process noise model
        BEGIN
            // 1. Compute wheel-speed variances via regression model
            var_vl ← MAX(k_vl * ABS(u_input.v_l) + var_base, var_base)
            var_vr ← MAX(k_vr * ABS(u_input.v_r) + var_base, var_base)

            // 2. Build input noise matrix
            Sigma_u ← DiagonalMatrix(var_vl, var_vr)

            // 3. Propagate through the dynamics
            G ← JacobianG(x_state, dt)
            Q_input ← G * Sigma_u * Transpose(G)

            // 4. Add small diagonal model noise for stability
            RETURN Q_input + Q_model_baseline
        END FUNCTION

END CLASS
```

### 5.7.2 Extended Kalman Filter Class

The EKF class maintains the current state estimate, its error covariance, and the measurement noise parameters. It performs a prediction step at every control cycle and a correction step only when valid camera measurements are available.

```text
CLASS EKF

    FUNCTION Initialize(model, x_init, P_init, R_matrix)
        BEGIN
            x ← x_init          // State estimate (3x1)
            P ← P_init          // Covariance estimate (3x3)
            R ← R_matrix        // Measurement noise (3x3)
            sys_model ← model
        END FUNCTION

    FUNCTION PredictStep(u_input)
        // Projects the state and covariance forward in time
        BEGIN
            // 1. Linearization and Noise Calculation
            F ← sys_model.JacobianDF(x, u_input, dt)
            Q ← sys_model.ComputeQ(x, u_input)

            // 2. Nonlinear Prediction
            x_pred ← sys_model.StateExtrapolation(x, u_input, dt)

            // 3. Covariance Propagation
            P_pred ← (F * P * Transpose(F)) + Q

            // Update internal state
            x ← x_pred
            P ← P_pred
        END FUNCTION

    FUNCTION UpdateStep(z_meas)
        // Corrects the predicted state using camera data
        BEGIN
            H ← IdentityMatrix(3)
            z ← Vector(z_meas)

            // 1. Compute Kalman Gain
            // S = H P H^T + R
            S ← (H * P * Transpose(H)) + R
            K ← P * Transpose(H) * Inverse(S)

            // 2. State Correction
            // x = x + K(z - Hx)
            innovation ← z - (H * x)
            x_new ← x + (K * innovation)

            // 3. Covariance Update (Joseph Form for numerical stability)
            // P = (I - KH) P (I - KH)^T + K R K^T
            I_KH ← (IdentityMatrix(3) - (K * H))
            P_new ← (I_KH * P * Transpose(I_KH)) + (K * R * Transpose(K))

            // Update internal state
            x ← x_new
            P ← P_new
            
            RETURN x, P
        END FUNCTION

END CLASS
```
### 5.7.3 EKF Cycle in the Main Control Loop

The EKF runs synchronously within the main control loop. The prediction step is executed at every timestep (dt), while the update step is event-triggered.

```text
// Main Control Loop
WHILE (SystemIsRunning) DO

    // 1. Acquire Sensor Data
    u_current.v_l ← GetMeasuredSpeedLeft()
    u_current.v_r ← GetMeasuredSpeedRight()

    // 2. EKF Prediction (Executed every cycle)
    ekf_instance.PredictStep(u_current)

    // 3. EKF Update (Conditional execution)
    IF (NewCameraPoseAvailable()) THEN
        z_cam ← GetCameraPose()
        ekf_instance.UpdateStep(z_cam)
    END IF

    // 4. Usage
    // Provide fused pose to navigation or path planning modules
    current_pose ← ekf_instance.x
    RunNavigationController(current_pose)

END WHILE

```
---

### 5.8 Obtained Regression Model's for Each Wheel (quite not a good fit, but that's what thymio offers)
The regression models for each wheel produced linear fits of the form $\operatorname{var}(u) \approx a|u| + b$. However, as expected for low-speed regions, the intercepts were negative (e.g., $-6.81$ for the left wheel and $-7.05$ for the right).

Since variances must remain non-negative for physical and numerical consistency, a lower bound (**base variance**) was introduced based on the smallest experimentally observed variance for each wheel:

* $\text{var}_{L,\text{base}} = 1.59$
* $\text{var}_{R,\text{base}} = 1.24$

During prediction, any value produced by the linear model that falls below these thresholds is replaced by the corresponding base variance. This **flooring mechanism** matches the behaviour implemented in `compute_Q()`, preventing the filter from collapsing at low speeds while retaining the essential trend encoded in the regression slopes ($0.289$ for the left wheel and $0.338$ for the right wheel), which capture how uncertainty grows with motor command.

<div style="display: flex; gap: 20px; justify-content: center; align-items: flex-start;">

  <figure style="margin: 0; text-align: center;">
    <img src="images/speed_variance.jpg" alt="speed_variance" width="300"/>
    <figcaption>Speed variance left wheel</figcaption>
  </figure>

  <figure style="margin: 0; text-align: center;">
    <img src="images/speed_varianceR.jpg" alt="speed_variance" width="300"/>
    <figcaption>Speed variance right wheel</figcaption>
  </figure>
</div>

![Example environment](images/table.jpg)

# Running the project

To run the project (after `git clone` and `pip install -r requirements.txt`), simply open a terminal, navigate to the project's `src` directory, and execute the main script using Python 3:

```bash
cd src
python main.py
```

# Conclusion
This project successfully implemented a complete autonomous navigation stack for the Thymio II robot, integrating computer vision, graph-based planning, sensor fusion, and reactive control into a cohesive system capable of navigating complex environments with both static and dynamic obstacles.

The **vision system** proved robust for environment mapping and real-time localization, with ArUco marker detection providing sub-centimeter accuracy. The visibility graph approach generated near-optimal paths through polygonal obstacle fields, while A* search efficiently computed collision-free trajectories. The **Extended Kalman Filter** effectively fused odometry and camera measurements, maintaining accurate pose estimates even during intermittent camera occlusions. The **hierarchical state machine controller** successfully balanced global path-following with local obstacle avoidance, demonstrating the value of combining deliberative planning with reactive behaviors.
However, the system revealed several limitations. 

## Limitations and Edges cases

### 1. Graph Node Obstruction
Placing a local obstacle directly on a visibility graph node will block that node, potentially making the path impossible to traverse. The system has no mechanism to detect or replan around occupied nodes in real-time.

### 2. Global Obstacle Collision During Local Avoidance
The reactive avoidance controller operates independently of the global map. Depending on the obstacle's position relative to the path, the Braitenberg behavior may steer the robot into a global obstacle (window). The controller has no awareness of the static map during the AVOIDING state

### 3. Initial Position Constraint
The robot's starting position must be at least ROBOT_RADIUS + cspace_padding (≈5.5 cm) away from any obstacle. If placed too close, the buffered obstacle polygons may engulf the start node, causing path planning to fail.

### 4. Goal Position in Obstacle
If the goal (red heart) is detected inside or too close to a buffered obstacle polygon during mapping, the goal node will be created but may have no valid edges in the visibility graph, making it unreachable. The system doesn't validate goal accessibility during initialization.

## References and Bibliography

Becker, G. (n.d.). *Kalman Filter from the Ground Up* [PDF]. Retrieved from https://kalmanfilter.net/book.html 

OpenCV ArUco Documentation: https://docs.opencv.org/4.x/d5/dae/tutorial_aruco_detection.html

NetworkX Documentation: https://networkx.org/

Shapely Documentation: https://shapely.readthedocs.io/

Thymio II Documentation: https://www.thymio.org/



## Acknowledgments

**Use of AI Tools:** Throughout this project, we utilized artificial intelligence assistants (ChatGPT, Gemini, Claude) as development tools for code debugging, algorithm explanation, documentation generation, and report writing assistance.